In [1]:
import pandas as pd
import numpy as np
import plotly.express as px # 3d plot
from collections import defaultdict


In [2]:
tdf_sub2 = pd.read_csv("tdata/tdf_sub2.csv",sep='\t', )


In [3]:
# fill null
tdf_sub2.with_who = tdf_sub2.with_who.fillna("missing_val")
tdf_sub2["where"] = tdf_sub2["where"].fillna("missing_val")


In [4]:
# create time col
tdf_sub2["Time"] = pd.to_datetime(tdf_sub2.timestamp_TD)
tdf_sub2.head()

,idcode,activity,with_who,where,mood,day_of_week,sex,latitude_home,longitude_home,timestamp_TD,Time
0,f88da12a9900972539558ad26ee4d65b48eef06f,Watcing Youtube Tv-shows etc.,Alone,Home Apartment Room,Fairly happy,Thursday,M,46.065852,11.119008,2018-05-10 00:02:18.790,2018-05-10 00:02:18.790
1,f88da12a9900972539558ad26ee4d65b48eef06f,Watcing Youtube Tv-shows etc.,Alone,Home Apartment Room,Fairly happy,Thursday,M,46.065852,11.119008,2018-05-10 00:32:18.635,2018-05-10 00:32:18.635
2,f88da12a9900972539558ad26ee4d65b48eef06f,Sleeping,Alone,Home Apartment Room,Fairly happy,Thursday,M,46.065852,11.119008,2018-05-10 01:02:18.641,2018-05-10 01:02:18.641
3,f88da12a9900972539558ad26ee4d65b48eef06f,Sleeping,Alone,Home Apartment Room,Fairly happy,Thursday,M,46.065852,11.119008,2018-05-10 01:32:18.620,2018-05-10 01:32:18.620
4,f88da12a9900972539558ad26ee4d65b48eef06f,Sleeping,Alone,Home Apartment Room,Fairly happy,Thursday,M,46.065852,11.119008,2018-05-10 02:02:18.631,2018-05-10 02:02:18.631


In [5]:
tdf_sub2.shape

(672, 11)

In [6]:
# How many days of data we have for sunject 1
tdf_sub2.Time.max() - tdf_sub2.Time.min()


Timedelta('13 days 23:29:59.210000')

# 2D Visualization

#### frequency of  activity   

In [10]:
df_a = pd.DataFrame(tdf_sub2.activity.value_counts()).reset_index(drop=False)
df_a.columns = ["activity","count"]
# df_a.head()

In [12]:

fig = px.bar(df_a, x="activity", y="count", color="activity", title="Subject_2 - frequency of  activity")
fig.show()


In [14]:
df_m = pd.DataFrame(tdf_sub2.mood.value_counts()).reset_index(drop=False)
df_m.columns = ["mood","count"]
# df_m.head()

In [15]:

fig = px.bar(df_m, x="mood", y="count", color="mood", title="Subject-2 frequency of  Mood")
fig.show()

In [17]:
df_f = pd.DataFrame(tdf_sub2.with_who.value_counts()).reset_index(drop=False)
df_f.columns = ["with_who","count"]
# df_f.head()

In [18]:

fig = px.bar(df_f, x="with_who", y="count", color="with_who", title="Subject-2 frequency of with-who data ")
fig.show()

In [20]:
df_l = pd.DataFrame(tdf_sub2["where"].value_counts()).reset_index(drop=False)
df_l.columns = ["where","count"]
# df_l.head()

In [21]:

fig = px.bar(df_l, x="where", y="count", color="where", title="Where subject-1 Frequently spent time ")
fig.show()

## 2D-  Activity  vs time of the day - [Heatmaps](https://plotly.com/python/heatmaps/)
* Given spcfic hour of the day find the frequency of activity


In [23]:
time_activity_freq = defaultdict(lambda: defaultdict(int))
for index,row in tdf_sub2.iterrows():
    # extract hour from the time-column
    hr = row.Time.strftime("%H")
    activity = row.activity
    time_activity_freq[hr][activity]+=1.0
# activity at time 13 of all-days 
time_activity_freq["13"] 

defaultdict(int,
            {'Eating': 7.0,
             'Free time': 5.0,
             'Null': 3.0,
             'Social media,Phone; chat': 1.0,
             'Travel': 4.0,
             'Lesson': 5.0,
             'Selfcare': 1.0,
             'Study': 2.0})

* give index for each activity 

In [25]:
activities = tdf_sub2.activity.unique().tolist()
activities.sort()
activity_to_index = {activity:i for i,activity in enumerate(activities)}
activity_to_index

{'Eating': 0,
 'Free time': 1,
 'Lesson': 2,
 'Null': 3,
 'Selfcare': 4,
 'Sleeping': 5,
 'Social life & Break': 6,
 'Social media,Phone; chat': 7,
 'Study': 8,
 'Travel': 9,
 'Watcing Youtube Tv-shows etc.': 10,
 'Work': 11}

* give index for each hour 

In [26]:

hours = list(time_activity_freq.keys())
hours.sort()
hour_to_index = {hour:i for i,hour in enumerate(hours)}
# hour_to_index

* Get numpy array of activity and hour


In [82]:
hit_map = np.zeros((len(activities),len(hours)))
for hr in time_activity_freq:
    for activity in time_activity_freq[hr]:
        hit_map[activity_to_index[activity]][hour_to_index[hr]]=time_activity_freq[hr][activity]
# hit_map       

In [83]:
fig = px.imshow(hit_map,
                labels=dict(y="Activity", x="Time of Day",color="count"),
                y=activities,
                x=hours,
                color_continuous_scale= 'blackbody'

             )
fig.update_layout(autosize = False, width=900,height = 600,title_text='Subject-2-the mapping of activity data with hours of the day')
fig.show()

## 2D- Mood  vs Time of the day 


In [29]:
time_mood_freq = defaultdict(lambda: defaultdict(int))
for index,row in tdf_sub2.iterrows():
    hr = row.Time.strftime("%H")
    mood = row.mood
    time_mood_freq[hr][mood]+=1.0
# activity at time 00 / midnight of the 13 days 
time_mood_freq["00"]

defaultdict(int,
            {'Fairly happy': 9.0,
             'Very happy': 10.0,
             'Fairly sad': 1.0,
             'Neutral': 1.0,
             'missing_val': 4.0,
             'Very sad': 3.0})

In [33]:
moods = tdf_sub2.mood.unique().tolist()
moods.sort()
# mood

In [34]:
mood_to_index = {mood:i for i,mood in enumerate(moods)}
# mood_to_index

In [63]:
# numpy array
hit_map = np.zeros((len(moods),len(hours)))
for hr in time_mood_freq:
    for mood in time_mood_freq[hr]:
        hit_map[mood_to_index[mood]][hour_to_index[hr]]=time_mood_freq[hr][mood]
# hit_map      

fig = px.imshow(hit_map,
                labels=dict(y="Mood", x="Time of Day",color="count"),
                y=moods,
                x=hours,
                color_continuous_scale= 'blackbody'
             )
fig.update_layout(autosize = False, width=900,height = 600,title_text='Subject-2-the mapping of mood data with hours of the day')

fig.show()

## 2D-   Mood vs Activity


In [38]:
mood_activity_freq =  defaultdict(lambda: defaultdict(int))
for index,row in tdf_sub2.iterrows():
    mood = row.mood
    activity = row.activity
    mood_activity_freq[mood][activity]+=1.0
# very happy with each activity count 
mood_activity_freq["Very happy"]

defaultdict(int,
            {'Social media,Phone; chat': 10.0,
             'Eating': 14.0,
             'Travel': 11.0,
             'Work': 7.0,
             'Watcing Youtube Tv-shows etc.': 2.0,
             'Social life & Break': 12.0,
             'Free time': 22.0,
             'Selfcare': 6.0,
             'Lesson': 12.0,
             'Sleeping': 6.0,
             'Study': 12.0})

In [66]:
# numpy array
hit_map = np.zeros((len(moods),len(activities)))
for mood in mood_activity_freq:
    for activity in mood_activity_freq[mood]:
        hit_map[mood_to_index[mood]][activity_to_index[activity]]=mood_activity_freq[mood][activity]
# hit_map      

fig = px.imshow(hit_map,
                labels=dict(y="Mood", x="Activity",color="count"),
                y=moods,
                x=activities,
                color_continuous_scale= 'blackbody'
             )
fig.update_layout(autosize = False, width=900,height = 600,title_text='subject_2- Mapping of mood data with activity ')
fig.show()

## 2D-   Mood vs With-Who


In [67]:
with_whos = tdf_sub2.with_who.unique().tolist()
with_whos.sort()


In [68]:
with_whos_to_index = {with_who:i for i,with_who in enumerate(with_whos)}
with_whos_to_index

{'Alone': 0,
 'Classmate(s)': 1,
 'Friend(s)': 2,
 'Relative(s)': 3,
 'Roommate(s)': 4,
 'missing_val': 5}

In [69]:
mood_with_who_freq =  defaultdict(lambda: defaultdict(int))
for index,row in tdf_sub2.iterrows():
    mood = row.mood
    with_who = row.with_who
    mood_with_who_freq[mood][with_who]+=1.0
# very happy with who she is count 
mood_with_who_freq["Very happy"]

defaultdict(int,
            {'Relative(s)': 7.0,
             'Alone': 64.0,
             'Friend(s)': 22.0,
             'Classmate(s)': 16.0,
             'Roommate(s)': 5.0})

In [70]:
# numpy array
hit_map = np.zeros((len(moods),len(with_whos)))
for mood in mood_with_who_freq:
    for with_who in mood_with_who_freq[mood]:
        hit_map[mood_to_index[mood]][with_whos_to_index[with_who]]=mood_with_who_freq[mood][with_who]
# hit_map      

fig = px.imshow(hit_map,
                labels=dict(y="Mood", x="with_who",color="count"),
                y=moods,
                x=with_whos,
                color_continuous_scale= 'blackbody'
             )
fig.update_layout(autosize = False, width=900,height = 600,title_text='subject_2- Mapping of mood data with with-who ')
fig.show()


## 2D-   week of the day vs Mood 


In [71]:
mood_days_of_week_freq =  defaultdict(lambda: defaultdict(int))
for index,row in tdf_sub2.iterrows():
    mood = row.mood
    week_day = row.day_of_week
    mood_days_of_week_freq[mood][week_day]+=1.0
# very happy with each day of the week  count (13 days)
mood_days_of_week_freq["Very happy"]

defaultdict(int,
            {'Thursday': 36.0,
             'Friday': 42.0,
             'Saturday': 14.0,
             'Sunday': 7.0,
             'Monday': 9.0,
             'Wednesday': 5.0,
             'Tuesday': 1.0})

In [72]:
days_of_weeks = tdf_sub2.day_of_week.unique().tolist()
days_of_weeks.sort()


In [73]:
day_of_week_to_index = {day_of_week:i for i,day_of_week in enumerate(days_of_weeks)}
day_of_week_to_index

{'Friday': 0,
 'Monday': 1,
 'Saturday': 2,
 'Sunday': 3,
 'Thursday': 4,
 'Tuesday': 5,
 'Wednesday': 6}

In [74]:
# numpy array
hit_map = np.zeros((len(moods),len(days_of_weeks)))
for mood in mood_days_of_week_freq:
    for day_of_week in mood_days_of_week_freq[mood]:
        hit_map[mood_to_index[mood]][day_of_week_to_index[day_of_week]]=mood_days_of_week_freq[mood][day_of_week]
# hit_map      

fig = px.imshow(hit_map,
                labels=dict(y="Mood", x="Day of week",color="count"),
                y=moods,
                x=days_of_weeks,
                color_continuous_scale= 'blackbody'
             )
fig.update_layout(autosize = False, width=900,height = 600,title_text='subject_2- Mapping of mood data with day of week ')
fig.show()

## 2D-   week of the day vs Activity  


In [75]:
activity_days_of_week_freq =  defaultdict(lambda: defaultdict(int))
for index,row in tdf_sub2.iterrows():
    activity = row.activity
    week_day = row.day_of_week
    activity_days_of_week_freq[activity][week_day]+=1.0
# counts of  activitis for monday   (13 days)
activity_days_of_week_freq["Work"]

defaultdict(int,
            {'Thursday': 3.0,
             'Friday': 7.0,
             'Monday': 3.0,
             'Tuesday': 1.0,
             'Wednesday': 3.0,
             'Saturday': 9.0,
             'Sunday': 3.0})

In [77]:
# numpy array
hit_map = np.zeros((len(days_of_weeks),len(activities)))
for day_of_week in activity_days_of_week_freq:
    for activity in activity_days_of_week_freq[day_of_week]:
        hit_map[day_of_week_to_index[day_of_week]][activity_to_index[activity]]=activity_days_of_week_freq[day_of_week][activity]
# hit_map      

fig = px.imshow(hit_map,
                labels=dict(y="Day of week", x="Activit",color="count"),
                y=days_of_weeks,
                x=activities,
                color_continuous_scale= 'blackbody'
             )
fig.update_layout(autosize = False, width=900,height = 600,title_text='subject_2- Mapping of week of day data with activity ')
fig.show()

KeyError: 'Watcing Youtube Tv-shows etc.'

# 4D  visualization 
## 4D visualization of mood 

In [59]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Trento")
location = geolocator.reverse("{},{}".format( tdf_sub2.latitude_home[0], tdf_sub2.longitude_home[0]))
location.raw["address"]

{'shop': 'Zanella',
 'house_number': '78',
 'road': 'Via Antonio Rosmini',
 'suburb': 'Centro storico Trento',
 'city': 'Trento',
 'municipality': "Territorio Val d'Adige",
 'county': 'Provincia di Trento',
 'state': 'Trentino-Alto Adige/Südtirol',
 'postcode': '38122',
 'country': 'Italia',
 'country_code': 'it'}

In [60]:
tdf_sub2["location_name"] = [location.raw["address"]["road"]]*len(tdf_sub2)
tdf_sub2.location_name[0]

'Via Antonio Rosmini'

In [79]:
# Plot the trajectory in space-time cube 

fig = px.line_3d(tdf_sub2, x="longitude_home", y="latitude_home", z="Time",color="mood",hover_name="location_name")
fig.update_traces(line=dict(width=5))
fig.update_layout(autosize = False,width = 800,height = 600,title_text='subject_2- 4D visualization(time-space-cube + mood)  ')


fig.show()

## 4D visualization , with who

In [80]:
# Plot the trajectory in space-time cube 

fig = px.line_3d(tdf_sub2, x="longitude_home", y="latitude_home", z="Time",color="with_who",hover_name="location_name")
fig.update_traces(line=dict(width=5))
fig.update_layout(autosize = False,width = 800,height = 600,title_text='subject_2- 4D visualization(time-space-cube + with_who)  ')



fig.show()